In [1]:
# coding=utf-8
import pandas as pd
import csv  # Import csv
import tweepy
import re
import string
import nltk
import pandas as pd
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sinling import SinhalaTokenizer, POSTagger
#pd.set_option('display.max_colwidth',1000)
tokenizer = SinhalaTokenizer()

In [2]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-

import codecs
df = pd.read_csv('FinalListPreprocessedData.csv', header = None, encoding='utf-8', engine='python')
df.columns = ['ID','User','CREATED_AT','LABEL','stemming_words','LIKE COUNT','FOLLOWERS COUNT']
df = df.dropna()
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොය...,1,1433
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා...,10,13898
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87


In [3]:
tokenizer = SinhalaTokenizer()
def clean_text(text):
    tokenized_sentences = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(text)]
    tagger = POSTagger()
    pos_tags =tagger.predict(tokenized_sentences) 
    return pos_tags 



In [4]:
df['cleaned_msg'] = df['stemming_words'].apply(lambda x: clean_text(x))
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,cleaned_msg
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොය...,1,1433,"[[(හාමුදුරුවරු, NNC), (සියලූම, NNC), (පූජකයන්ග..."
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා...,10,13898,"[[(ද්විපාර්ශවික, NNP), (එකඟතා, NNP), (ජන, JJ),..."
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ..."
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ..."
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ..."


In [5]:
def pos_tag(data):
    result = []
    if type(data) == tuple:
        return data[1]

    if type(data) == list:
        for inner in data:
            result.append(pos_tag(inner))
    return result 

In [6]:
df['pos_tagging'] = df['cleaned_msg'].apply(lambda x: pos_tag(x))
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,cleaned_msg,pos_tagging
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොය...,1,1433,"[[(හාමුදුරුවරු, NNC), (සියලූම, NNC), (පූජකයන්ග...","[[NNC, NNC, NNC, JJ, NNC, VNF, VNN, NNJ, NNC, ..."
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා...,10,13898,"[[(ද්විපාර්ශවික, NNP), (එකඟතා, NNP), (ජන, JJ),...","[[NNP, NNP, JJ, NNJ, NNC, RB, JCV, VNF, AUX, J..."
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, POST, NNP, NNC, C..."
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, RRPCV, VFM, FS]]"
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, POST, FS]]"


In [7]:
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import re

# My work

In [70]:
def get_pos_vec(x):
    w2v_model = Word2Vec(min_count=1,window=2,vector_size=5,sample=1e-5,alpha=0.01,min_alpha=0.0007,negative=0,workers=2)
    w2v_model.build_vocab(x, progress_per=1)
    w2v_model.train(x, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)
    
    for index, key in enumerate(w2v_model.wv.key_to_index):
        vec = w2v_model.wv[key]
    return vec

In [71]:
df['pos_vec'] = df['pos_tagging'].apply(lambda x: get_pos_vec(x))

In [72]:
def get_word_vec(text):
    x = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(text)]
    w2v_model = Word2Vec(min_count=1,window=2,vector_size=5,sample=1e-5,alpha=0.01,min_alpha=0.0007,negative=0,workers=2)
    w2v_model.build_vocab(x, progress_per=1)
    w2v_model.train(x, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)
    
    for index, key in enumerate(w2v_model.wv.key_to_index):
        vec = w2v_model.wv[key]
    return vec

In [73]:
df['word_vec'] = df['stemming_words'].apply(lambda x: get_word_vec(x))

In [74]:
#get sum of columns

dflist = ['pos_vec','word_vec']
df['sums'] = df[dflist].sum(axis=1)

In [75]:
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,cleaned_msg,pos_tagging,pos_vec,word_vec,sums
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොය...,1,1433,"[[(හාමුදුරුවරු, NNC), (සියලූම, NNC), (පූජකයන්ග...","[[NNC, NNC, NNC, JJ, NNC, VNF, VNN, NNJ, NNC, ...","[-0.15023164, -0.018600846, 0.19076237, -0.146...","[-0.16315837, 0.089915946, -0.08274154, 0.0164...","[-0.31339002, 0.0713151, 0.10802083, -0.129892..."
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා...,10,13898,"[[(ද්විපාර්ශවික, NNP), (එකඟතා, NNP), (ජන, JJ),...","[[NNP, NNP, JJ, NNJ, NNC, RB, JCV, VNF, AUX, J...","[-0.192071, 0.10014586, -0.17519173, -0.087836...","[-0.08924355, 0.09035001, -0.13573924, -0.0709...","[-0.28131455, 0.19049588, -0.31093097, -0.1588..."
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, POST, NNP, NNC, C...","[-0.038754843, 0.1615487, -0.118617915, 0.0009...","[0.049395896, -0.017760564, 0.110673234, -0.05...","[0.010641053, 0.14378814, -0.007944681, -0.053..."
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, RRPCV, VFM, FS]]","[0.14623532, 0.10140524, 0.13515386, 0.0152573...","[-0.192071, 0.10014586, -0.17519173, -0.087836...","[-0.04583569, 0.20155111, -0.04003787, -0.0725..."
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87,"[[(කරැනාකර, NNC), (චින, VP), (නිෂ්පාදිත, VP), ...","[[NNC, VP, VP, NNC, VP, NNC, POST, FS]]","[-0.03632035, 0.057531595, 0.019837474, -0.165...","[-0.038754843, 0.1615487, -0.118617915, 0.0009...","[-0.075075194, 0.2190803, -0.09878044, -0.1648..."


In [76]:
X = df['sums'].to_numpy()
X = X.reshape(-1,1)

In [77]:
X.shape

(532, 1)

In [78]:
# 5 kiyana eka vector size ekath ekka wenas wenawa
X = np.concatenate(np.concatenate(X, axis = 0),axis = 0).reshape(-1,5)

In [79]:
y=df['LABEL']

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [81]:
X_train.shape, X_test.shape

((425, 5), (107, 5))

In [82]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train,y_train)

LinearSVC()

In [83]:
y_pred = clf.predict(X_test)

In [84]:
from sklearn.metrics import classification_report
from sklearn import metrics

print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.classification_report(y_test,y_pred))



[[ 0  0 14  4  0]
 [ 0  0  2  2  0]
 [ 0  0 34  8  0]
 [ 0  0 16  6  0]
 [ 0  0 16  5  0]]
               precision    recall  f1-score   support

        Crime       0.00      0.00      0.00        18
Entertainment       0.00      0.00      0.00         4
        Other       0.41      0.81      0.55        42
     Politics       0.24      0.27      0.26        22
        Sport       0.00      0.00      0.00        21

     accuracy                           0.37       107
    macro avg       0.13      0.22      0.16       107
 weighted avg       0.21      0.37      0.27       107



c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chandima samarakoon\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

In [25]:
# from gensim.test.utils import common_texts
from gensim.models import Word2Vec

pos_tag_list = [['DT', 'JJ', 'NN', 'NN', 'VBD', 'IN', 'DT', 'NN'],
                      ['DT','NN','VBD','IN','DT','NN'],['PRP','VBP','JJ','JJ','NNS']]
w2v_model = Word2Vec(min_count=1,
                 window=2,
                 vector_size=2,
                 sample=1e-5, 
                 alpha=0.01, 
                 min_alpha=0.0007, 
                 negative=0,
                 workers=2)
w2v_model.build_vocab(pos_tag_list, progress_per=1)
w2v_model.train(pos_tag_list, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)

# get the vectors for the Pos_tags from w2v_model
my_dict = dict({})
for index, key in enumerate(w2v_model.wv.key_to_index):
    my_dict[key] = w2v_model.wv[key]

print(my_dict)

{'NN': array([-0.02681136,  0.01182151], dtype=float32), 'DT': array([0.25516748, 0.45046365], dtype=float32), 'JJ': array([-0.4651475 , -0.35584044], dtype=float32), 'IN': array([0.32294357, 0.4486494 ], dtype=float32), 'VBD': array([-0.2507714 , -0.18816864], dtype=float32), 'NNS': array([ 0.36902523, -0.07667363], dtype=float32), 'VBP': array([-0.22683072,  0.32770252], dtype=float32), 'PRP': array([-0.24300802, -0.09080088], dtype=float32)}
